In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
# !pip install pyspellchecker
from spellchecker import SpellChecker
from nltk.corpus import stopwords
import spacy
nlp = spacy.load('en_core_web_lg')
import string
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
from sklearn.metrics import confusion_matrix , classification_report
from sklearn.metrics.classification import accuracy_score , log_loss
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier

from sklearn.ensemble import VotingClassifier

C:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)
from sklearn.model_selection import RandomizedSearchCV

In [10]:
train = pd.read_csv(r"C:\Users\admin\Desktop\Datasets\NLP Disaster Tweets\train.csv")
test = pd.read_csv(r"C:\Users\admin\Desktop\Datasets\NLP Disaster Tweets\test.csv")

In [11]:
train.drop('id' , axis=1 , inplace=True)
train.head()

,keyword,location,text,target
0,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [12]:
test.drop('id' , axis=1 , inplace=True)
test.head()

,keyword,location,text
0,NaN,NaN,Just happened a terrible car crash
1,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [13]:
print(train.shape)
print(test.shape)

(7613, 4)
(3263, 3)


In [14]:
train.isnull().sum()

keyword       61
location    2533
text           0
target         0
dtype: int64

In [15]:
test.isnull().sum()

keyword       26
location    1105
text           0
dtype: int64

In [16]:
# Finding space filled text data in train data
lst=[]
for i,key,loc,txt,target in train.itertuples():
      if type(txt) == str:
            if txt.isspace():
                  lst.append(i)
lst
# Hence no space filled data

[]

In [17]:
# Finding space filled text data in test data
lst=[]
for i,key,loc,txt in test.itertuples():
      if type(txt) == str:
            if txt.isspace():
                  lst.append(i)
lst
# Hence no space filled data

[]

In [18]:
# Data Cleaning

In [19]:
data = pd.concat([train["text"] , test["text"]])
data = pd.DataFrame({"text" : data})
print(data.shape)
data.head()

(10876, 1)


,text
0,Our Deeds are the Reason of this #earthquake M...
1,Forest fire near La Ronge Sask. Canada
2,All residents asked to 'shelter in place' are ...
3,"13,000 people receive #wildfires evacuation or..."
4,Just got sent this photo from Ruby #Alaska as ...


In [20]:
# Removing URls
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

# Remove HTML Tags
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

# Remove Emojis
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                                "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# Remove Punctuation
def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

# Lower
def lower(text):
    text = text.lower()
    return text

# Stopwords
def stop_word(text):
    text = text.split()
    text = [word for word in text if not word in set(stopwords.words('english'))]
    text = " ".join(text)
    return (text)

# Spelling Correction
spell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return (" ".join(corrected_text))

In [21]:
data['text'] = data['text'].apply(lambda x : remove_URL(x))
data['text'] = data['text'].apply(lambda x : remove_html(x))
data['text'] = data['text'].apply(lambda x : remove_emoji(x))
data['text'] = data['text'].apply(lambda x : lower(x))
data['text'] = data['text'].apply(lambda x : stop_word(x))
data['text'] = data['text'].apply(lambda x : remove_punct(x))
#data['text'] = data['text'].apply(lambda x : correct_spellings(x))

In [22]:
train.drop("text" , axis=1 , inplace=True)
test.drop("text" , axis=1 , inplace=True)

train_txt = data.iloc[:7613,-1]
test_txt = data.iloc[7613:,-1]

train_txt = pd.DataFrame({"text" : train_txt})
test_txt = pd.DataFrame({"text" : test_txt})

train = pd.concat([train , train_txt] , axis=1)
train = train[["keyword" , "location" , "text" , "target"]]
test = pd.concat([test , test_txt] , axis=1)

In [23]:
train.head()

,keyword,location,text,target
0,NaN,NaN,deeds reason earthquake may allah forgive us,1
1,NaN,NaN,forest fire near la ronge sask canada,1
2,NaN,NaN,residents asked shelter place notified officer...,1
3,NaN,NaN,13000 people receive wildfires evacuation orde...,1
4,NaN,NaN,got sent photo ruby alaska smoke wildfires pou...,1


In [24]:
test.head()

,keyword,location,text
0,NaN,NaN,happened terrible car crash
1,NaN,NaN,heard earthquake different cities stay safe ev...
2,NaN,NaN,forest fire spot pond geese fleeing across str...
3,NaN,NaN,apocalypse lighting spokane wildfires
4,NaN,NaN,typhoon soudelor kills 28 china taiwan


In [25]:
# Spacy stopword and lemmetization 
# Train Data
corpus_train = []
for i in range (len(train)):
    review = train.loc[i , "text"]
    review = nlp(review)
    review = [token.lemma_ for token in review]
    review = " ".join(review)
    corpus_train.append(review)

In [26]:
# Spacy stopword and lemmetization 
# Text Data
corpus_test = []
for i in range (len(test)):
    review = test.loc[i , "text"]
    review = nlp(review)
    review = [token.lemma_ for token in review]
    review = " ".join(review)
    corpus_test.append(review)

In [27]:
# Putting this new clean text to original file
# Train Data
Text = pd.DataFrame ( { "text" : corpus_train })
train.drop(["text"],inplace=True,axis=1)
train = pd.concat ( [train , Text] , axis=1)
train = train[["keyword" , "location" , "text" , "target"]]
train.head()

,keyword,location,text,target
0,NaN,NaN,deed reason earthquake may allah forgive -PRON-,1
1,NaN,NaN,forest fire near la ronge sask canada,1
2,NaN,NaN,resident ask shelter place notify officer evac...,1
3,NaN,NaN,13000 people receive wildfire evacuation order...,1
4,NaN,NaN,get send photo ruby alaska smoke wildfire pour...,1


In [28]:
# Test Data
Text = pd.DataFrame ( { "text" : corpus_test })
test.drop(["text"],inplace=True,axis=1)
test = pd.concat ( [test , Text] , axis=1)
test.head()

,keyword,location,text
0,NaN,NaN,happen terrible car crash
1,NaN,NaN,hear earthquake different city stay safe everyone
2,NaN,NaN,forest fire spot pond goose flee across street...
3,NaN,NaN,apocalypse lighting spokane wildfire
4,NaN,NaN,typhoon soudelor kill 28 china taiwan


In [29]:
# Removing data for "-.....-" words such as -PRON-
for i in range(len(train)):
    txt = train.loc[i,"text"]
    txt = re.sub(' -\w+-' , "" , txt)
    train.loc[i , "text"] = txt

for i in range(len(test)):
    txt = test.loc[i,"text"]
    txt = re.sub(' -\w+-' , "" , txt)
    test.loc[i , "text"] = txt

In [30]:
train.head()

,keyword,location,text,target
0,NaN,NaN,deed reason earthquake may allah forgive,1
1,NaN,NaN,forest fire near la ronge sask canada,1
2,NaN,NaN,resident ask shelter place notify officer evac...,1
3,NaN,NaN,13000 people receive wildfire evacuation order...,1
4,NaN,NaN,get send photo ruby alaska smoke wildfire pour...,1


In [31]:
test.head()

,keyword,location,text
0,NaN,NaN,happen terrible car crash
1,NaN,NaN,hear earthquake different city stay safe everyone
2,NaN,NaN,forest fire spot pond goose flee across street...
3,NaN,NaN,apocalypse lighting spokane wildfire
4,NaN,NaN,typhoon soudelor kill 28 china taiwan


In [32]:
train["keyword"].fillna("" , inplace=True)
train["location"].fillna("" , inplace=True)

In [33]:
test["keyword"].fillna("" , inplace=True)
test["location"].fillna("" , inplace=True)

In [34]:
train.isnull().sum()

keyword     0
location    0
text        0
target      0
dtype: int64

In [35]:
test.isnull().sum()

keyword     0
location    0
text        0
dtype: int64

In [36]:
# Combining keyword to text
train["final_text"] = train["keyword"] + " " + train["text"]
train = train[["keyword" , "location" , "text" , "final_text" , "target"]]
test["final_text"] = test["keyword"] + " " + test["text"]

In [37]:
train.head()

,keyword,location,text,final_text,target
0,,,deed reason earthquake may allah forgive,deed reason earthquake may allah forgive,1
1,,,forest fire near la ronge sask canada,forest fire near la ronge sask canada,1
2,,,resident ask shelter place notify officer evac...,resident ask shelter place notify officer eva...,1
3,,,13000 people receive wildfire evacuation order...,13000 people receive wildfire evacuation orde...,1
4,,,get send photo ruby alaska smoke wildfire pour...,get send photo ruby alaska smoke wildfire pou...,1


In [38]:
test.head()

,keyword,location,text,final_text
0,,,happen terrible car crash,happen terrible car crash
1,,,hear earthquake different city stay safe everyone,hear earthquake different city stay safe ever...
2,,,forest fire spot pond goose flee across street...,forest fire spot pond goose flee across stree...
3,,,apocalypse lighting spokane wildfire,apocalypse lighting spokane wildfire
4,,,typhoon soudelor kill 28 china taiwan,typhoon soudelor kill 28 china taiwan


In [39]:
print(train.shape)
print(test.shape)

(7613, 5)
(3263, 4)


In [41]:
# Saving Data
train.to_csv(r"C:\Users\admin\Desktop\Datasets\NLP Disaster Tweets\train_clean.csv" , index=False , header=True)
test.to_csv(r"C:\Users\admin\Desktop\Datasets\NLP Disaster Tweets\test_clean.csv" , index=False , header=True)

In [4]:
# Import clean data
train = pd.read_csv(r"C:\Users\admin\Desktop\Datasets\NLP Disaster Tweets\train_clean.csv")
test = pd.read_csv(r"C:\Users\admin\Desktop\Datasets\NLP Disaster Tweets\test_clean.csv")

In [5]:
train.head()

,keyword,location,text,final_text,target
0,NaN,NaN,deed reason earthquake may allah forgive,deed reason earthquake may allah forgive,1
1,NaN,NaN,forest fire near la ronge sask canada,forest fire near la ronge sask canada,1
2,NaN,NaN,resident ask shelter place notify officer evac...,resident ask shelter place notify officer eva...,1
3,NaN,NaN,13000 people receive wildfire evacuation order...,13000 people receive wildfire evacuation orde...,1
4,NaN,NaN,get send photo ruby alaska smoke wildfire pour...,get send photo ruby alaska smoke wildfire pou...,1


In [6]:
test.head()

,keyword,location,text,final_text
0,NaN,NaN,happen terrible car crash,happen terrible car crash
1,NaN,NaN,hear earthquake different city stay safe everyone,hear earthquake different city stay safe ever...
2,NaN,NaN,forest fire spot pond goose flee across street...,forest fire spot pond goose flee across stree...
3,NaN,NaN,apocalypse lighting spokane wildfire,apocalypse lighting spokane wildfire
4,NaN,NaN,typhoon soudelor kill 28 china taiwan,typhoon soudelor kill 28 china taiwan


In [7]:
# Combining data
data = pd.concat([train["final_text"] , test["final_text"]])
data = pd.DataFrame({"text" : data})
data.head()

,text
0,deed reason earthquake may allah forgive
1,forest fire near la ronge sask canada
2,resident ask shelter place notify officer eva...
3,13000 people receive wildfire evacuation orde...
4,get send photo ruby alaska smoke wildfire pou...


In [8]:
data.shape

(10876, 1)

In [9]:
# Tfidf vectorizer

In [10]:
vectorizer = TfidfVectorizer(ngram_range=(1,2))
data_vec = vectorizer.fit_transform(data["text"])
data_vec

<10876x92952 sparse matrix of type '<class 'numpy.float64'>'
	with 208482 stored elements in Compressed Sparse Row format>

In [11]:
train_vec , test_vec = train_test_split(data_vec , train_size=7613 , shuffle=False)

In [12]:
train_vec

<7613x92952 sparse matrix of type '<class 'numpy.float64'>'
	with 145373 stored elements in Compressed Sparse Row format>

In [13]:
test_vec

<3263x92952 sparse matrix of type '<class 'numpy.float64'>'
	with 63109 stored elements in Compressed Sparse Row format>

In [14]:
print(train_vec.shape)
print(test_vec.shape)

(7613, 92952)
(3263, 92952)


In [15]:
x_train , x_test , y_train , y_test = train_test_split (train_vec , train["target"] , test_size=0.3 , random_state=42)

In [16]:
print (x_train.shape)
print (y_train.shape)

print (x_test.shape)
print (y_test.shape)

(5329, 92952)
(5329,)
(2284, 92952)
(2284,)


In [17]:
# MultinomialNB
clf = MultinomialNB(alpha=0.6)
clf.fit (x_train , y_train)
prediction1 = clf.predict ( test_vec )
metrics.accuracy_score ( y_test , clf.predict( x_test ) )

0.7999124343257443

In [18]:
# Support Vector Machine

In [28]:
hyperparameter_grid={
    'C'                           : [None , 0.0001 , 0.001 , 0.01 , 1 , 2 , 5 , 7 , 10 , 50 , 100 , 500 , 1000] , 
    'gamma'                       : ['scale', 'auto' , 1, 0.1, 0.01, 0.001, 0.0001 , 0.00001 , 0.000001] , 
    'kernel'                      : ['rbf' , 'sigmoid' , 'poly'],
    'shrinking'                   : [True , False] ,
    'probability'                 : [True , False , None] ,
    'class_weight'                : ['balanced', None] ,
    'decision_function_shape'     : ['ovo', 'ovr'],
    'break_ties'                  : [True , False],
    'random_state'                : [42]
}

In [29]:
clf = SVC()
random_cv = RandomizedSearchCV(estimator=clf , param_distributions = hyperparameter_grid , 
            cv=5 , scoring = 'roc_auc',n_jobs = -1, verbose = 5,random_state=42)
random_cv.fit(x_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  35 out of  50 | elapsed:   21.0s remaining:    9.0s
[Parallel(n_jobs=-1)]: Done  46 out of  50 | elapsed:   52.9s remaining:    4.5s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   56.0s finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                 class_weight=None, coef0=0.0,
                                 decision_function_shape='ovr', degree=3,
                                 gamma='scale', kernel='rbf', max_iter=-1,
                                 probability=False, random_state=None,
                                 shrinking=True, tol=0.001, verbose=False),
                   iid='deprecated', n_iter=10, n_jobs=-1,
                   param_distributions={'C': [None, 0.0001, 0...
                                        'class_weight': ['balanced', None],
                                        'decision_function_shape': ['ovo',
                                                                    'ovr'],
                                        'gamma': ['scale', 'auto', 1, 0.1, 0.01,
                                                  0.001, 0.0001, 1e-05, 1e-06],
             

In [30]:
random_cv.best_estimator_

SVC(C=1, break_ties=True, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='sigmoid',
    max_iter=-1, probability=False, random_state=42, shrinking=True, tol=0.001,
    verbose=False)

In [18]:
clf = SVC(C=1, break_ties=True, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='sigmoid',
    max_iter=-1, probability=False, random_state=42, shrinking=True, tol=0.001,
    verbose=False)
clf.fit(x_train,y_train)
prediction2 = clf.predict(test_vec)

In [32]:
score_train = cross_val_score(clf, x_train,y_train, cv=k_fold, n_jobs=-1, scoring="accuracy")
print (round(np.mean(score_train)*100,2))
score_test = cross_val_score(clf, x_test,y_test, cv=k_fold, n_jobs=-1, scoring="accuracy")
print (round(np.mean(score_test)*100,2))

79.4
77.63


In [33]:
# KNN

In [34]:
hyperparameter_grid={
    'n_neighbors'        : [1 , 2 , 3 , 4 , 5 , 6 , 7 , 9 , 10 , 11 , 13 , 15 , 17 , 20 , 23 , 25 , 27 , 30 , 40 , 50] , 
    'weights'            : ['uniform' , 'distance'] , 
    'algorithm'          : ['auto', 'ball_tree', 'kd_tree', 'brute'] , 
    'leaf_size'          : [None , 1 , 2 , 3 , 5 , 7 , 9 , 10 , 20 , 30 , 40 , 50 , 60 , 70 , 80 , 90 , 100] , 
    'p'                  : [1 , 2 , 3] ,
    'n_jobs'             : [-1]
}

In [35]:
clf = KNeighborsClassifier()
random_cv = RandomizedSearchCV(estimator=clf , param_distributions = hyperparameter_grid ,
            cv=5 , n_iter = 50 , scoring = 'roc_auc',n_jobs = -1,verbose = 5,random_state=42)
random_cv.fit(x_train , y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 178 tasks      | elapsed:   16.2s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   22.6s finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=KNeighborsClassifier(algorithm='auto',
                                                  leaf_size=30,
                                                  metric='minkowski',
                                                  metric_params=None,
                                                  n_jobs=None, n_neighbors=5,
                                                  p=2, weights='uniform'),
                   iid='deprecated', n_iter=50, n_jobs=-1,
                   param_distributions={'algorithm': ['auto', 'ball_tree',
                                                      'kd_tree', 'brute'],
                                        'leaf_size': [None, 1, 2, 3, 5, 7, 9,
                                                      10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100],
                                        'n_jobs': [-1],
                                        

In [36]:
random_cv.best_estimator_

KNeighborsClassifier(algorithm='brute', leaf_size=None, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=40, p=2,
                     weights='distance')

In [19]:
clf = KNeighborsClassifier(algorithm='brute', leaf_size=None, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=40, p=2,
                     weights='distance')    
clf.fit(x_train , y_train)
prediction3 = clf.predict(test_vec)

In [38]:
score_train = cross_val_score(clf, x_train,y_train, cv=k_fold, n_jobs=-1, scoring="accuracy")
print (round(np.mean(score_train)*100,2))
score_test = cross_val_score(clf, x_test,y_test, cv=k_fold, n_jobs=-1, scoring="accuracy")
print (round(np.mean(score_test)*100,2))

76.17
74.39


In [39]:
# Logistic Regression

In [40]:
hyperparameter_grid = {
    'penalty'            : ['l1', 'l2', 'elasticnet'],
    'C'                  : [None , 0.0001 , 0.001 , 0.01 , 1 , 2 , 5 , 6 , 7 , 10 , 50 , 100 , 500 , 1000 , 10000] ,  
    'fit_intercept'      : [True , False],
    'solver'             : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter'           : [50 , 100 , 200 , 300 , 400 , 500],
    'multi_class'        : ['auto', 'ovr', 'multinomial'],
    'warm_start'         : [True],
    'n_jobs'             : [-1],
    'class_weight'       : ["balanced" , None]
    }

In [41]:
clf = LogisticRegression()
random_cv = RandomizedSearchCV(estimator=clf , param_distributions = hyperparameter_grid , n_iter=50 , 
            cv=5 , scoring = 'roc_auc',n_jobs = -1, verbose = 5 ,random_state=42)
random_cv.fit(x_train,y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 114 tasks      | elapsed:   14.5s
[Parallel(n_jobs=-1)]: Done 222 tasks      | elapsed:   55.8s
[Parallel(n_jobs=-1)]: Done 235 out of 250 | elapsed:   57.3s remaining:    3.6s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  1.9min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=LogisticRegression(C=1.0, class_weight=None,
                                                dual=False, fit_intercept=True,
                                                intercept_scaling=1,
                                                l1_ratio=None, max_iter=100,
                                                multi_class='auto', n_jobs=None,
                                                penalty='l2', random_state=None,
                                                solver='lbfgs', tol=0.0001,
                                                verbose=0, warm_start=False),
                   iid='deprecated', n_iter=50, n_jobs=-1,
                   param_distributions={'C': [N...
                                        'class_weight': ['balanced', None],
                                        'fit_intercept': [True, False],
                                        'max_iter': [50, 100, 200, 300, 400,
       

In [42]:
random_cv.best_estimator_

LogisticRegression(C=2, class_weight=None, dual=False, fit_intercept=False,
                   intercept_scaling=1, l1_ratio=None, max_iter=400,
                   multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=True)

In [20]:
clf = LogisticRegression(C=2, class_weight=None, dual=False, fit_intercept=False,
                   intercept_scaling=1, l1_ratio=None, max_iter=400,
                   multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=True)    
clf.fit(x_train , y_train)
prediction4 = clf.predict(test_vec)

In [44]:
score_train = cross_val_score(clf, x_train,y_train, cv=k_fold, n_jobs=-1, scoring="accuracy")
print (round(np.mean(score_train)*100,2))
score_test = cross_val_score(clf, x_test,y_test, cv=k_fold, n_jobs=-1, scoring="accuracy")
print (round(np.mean(score_test)*100,2))

79.19
76.79


In [21]:
# SGDClassifier
clf = SGDClassifier()
clf.fit (x_train , y_train)
prediction5 = clf.predict ( test_vec )
metrics.accuracy_score ( y_test , clf.predict( x_test ) )

0.8016637478108581

In [46]:
# Decision Tree Classifier

In [47]:
hyperparameter_grid={
    'criterion'                 : ["gini" , "entropy"],
    'splitter'                  : ["best" , "random" ],
    'min_samples_split'         : [1 , 2 , 3],
    'min_samples_leaf'          : [1 , 2 , 3 , 5 , 7 , 9 , 11 , 13 , 15 , 17 , 19 , 21],
    'max_features'              : [1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 ,"auto", "sqrt", "log2"],
    'class_weight'              : ['balanced' , None]
}

In [50]:
clf = DecisionTreeClassifier()
random_cv = RandomizedSearchCV(estimator=clf , param_distributions = hyperparameter_grid ,
            cv=10 , n_iter = 50 , scoring = 'roc_auc',n_jobs = -1,verbose = 5,random_state=42)
random_cv.fit(x_train , y_train)

Fitting 10 folds for each of 50 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 188 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    8.6s finished


RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=None,
         

In [51]:
random_cv.best_estimator_

DecisionTreeClassifier(ccp_alpha=0.0, class_weight='balanced',
                       criterion='entropy', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       presort='deprecated', random_state=None,
                       splitter='best')

In [22]:
clf = DecisionTreeClassifier(ccp_alpha=0.0, class_weight='balanced',
                       criterion='entropy', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       presort='deprecated', random_state=None,
                       splitter='best')
clf.fit(x_train , y_train)
prediction6 = clf.predict(test_vec)

In [23]:
score_train = cross_val_score(clf, x_train,y_train, cv=k_fold, n_jobs=-1, scoring="accuracy")
print (round(np.mean(score_train)*100,2))
score_test = cross_val_score(clf, x_test,y_test, cv=k_fold, n_jobs=-1, scoring="accuracy")
print (round(np.mean(score_test)*100,2))

71.4
69.18


In [24]:
# Random Forest Classifier

In [25]:
clf = RandomForestClassifier()
clf.fit (x_train , y_train)
prediction7 = clf.predict ( test_vec )
metrics.accuracy_score ( y_test , clf.predict( x_test ) )

0.7723292469352014

In [67]:
# XGBoost

In [26]:
clf = xgb.XGBClassifier()
clf.fit(x_train , y_train)
prediction8 = clf.predict(test_vec)

In [27]:
score_train = cross_val_score(clf, x_train,y_train, cv=k_fold, n_jobs=-1, scoring="accuracy")
print ( round(np.mean(score_train)*100,2) )
score_test = cross_val_score(clf, x_test,y_test, cv=k_fold, n_jobs=-1, scoring="accuracy")
print ( round(np.mean(score_test)*100,2) )

72.83
70.36


In [29]:
# Light Boost Classifier
clf = lgb.LGBMClassifier()
clf.fit (x_train , y_train)
prediction9 = clf.predict ( test_vec )
metrics.accuracy_score ( y_test , clf.predict( x_test ) )

0.7780210157618214

In [31]:
# Bagging Classifier
clf = BaggingClassifier()
clf.fit (x_train , y_train)
prediction10 = clf.predict ( test_vec )
metrics.accuracy_score ( y_test , clf.predict( x_test ) )

0.7644483362521891

In [33]:
# Gradient Boosting Classifier
clf = GradientBoostingClassifier()
clf.fit (x_train , y_train)
prediction11 = clf.predict ( test_vec )
metrics.accuracy_score ( y_test , clf.predict( x_test ) )

0.7368651488616462

In [35]:
# AdaBoost Classifier
clf = AdaBoostClassifier()
clf.fit (x_train , y_train)
prediction12 = clf.predict ( test_vec )
metrics.accuracy_score ( y_test , clf.predict( x_test ) )

0.7338003502626971

In [37]:
# Extra Trees Classifier
clf = ExtraTreesClassifier()
clf.fit (x_train , y_train)
prediction13 = clf.predict ( test_vec )
metrics.accuracy_score ( y_test , clf.predict( x_test ) )

0.7810858143607706

In [87]:
# Ensemble Voting

sub=[]
sub.append(prediction1)
sub.append(prediction2)
#sub.append(prediction3)
sub.append(prediction4)
sub.append(prediction5)
#sub.append(prediction6)
sub.append(prediction7)
#sub.append(prediction8)
#sub.append(prediction9)
sub.append(prediction10)
#sub.append(prediction11)
#sub.append(prediction12)
sub.append(prediction13)

sub = np.array(sub)
sub_final = []
for i in range (len(sub[0])):
    p = 0
    n = 0
    for j in range (len(sub)):
        if (sub[j][i]==1):
            p=p+1
        else:
            n=n+1
    if (p>n):
        sub_final.append(1)
    else:
        sub_final.append(0)

In [45]:
test = pd.read_csv(r"C:\Users\admin\Desktop\Datasets\NLP Disaster Tweets\sample_submission.csv")
submission = pd.DataFrame({"id": test["id"] , "target": sub_final})

submission["id"] = submission["id"].astype(int)
submission["target"] = submission["target"].astype(int)

submission.to_csv(r"C:\Users\admin\Desktop\submission_tweet.csv" , index=False , header=True)